In [1]:
%load_ext autoreload
%autoreload 2
# this only works on startup!
from jax import config
config.update("jax_enable_x64", True)

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = '3'



from encoding_information.gpu_utils import limit_gpu_memory_growth
limit_gpu_memory_growth()

from cleanplots import *
from tqdm import tqdm
from signal_utils_1D import *
from plot_utils_1D import *

2024-10-14 09:35:49.324540: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-14 09:35:50.442741: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-10-14 09:35:50.442878: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-10-14 09:35:50.442892: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [13]:
from scipy.signal import resample

N_objects = 5000
noise_sigma = 1e-3
num_nyquist_samples_list = [4, 9, 16, 25, 36]
max_epochs = 5000
confidence = .95
num_replicates = 2

object_names = ['white_noise', 'delta_uniform', '8_deltas_uniform']

num_nyquist_samples = NUM_NYQUIST_SAMPLES


def white_noise_fn():
  # White noise object with 2x the bandwidth of the highest Nyquist frequency
  objects = []
  for i in range(N_objects):  
    obj = generate_random_object('white_noise', object_size=2*num_nyquist_samples_list[-1])
    # upsample it
    obj = resample(obj, UPSAMPLED_SIGNAL_LENGTH)
    # make positive and sum to 1
    obj -= np.min(obj)
    obj /= np.sum(obj)
    objects.append(obj)
  # print(objects[0].shape)
  return np.array(objects) 

delta_fn = lambda : np.array([generate_random_object('delta', num_deltas=1, object_size=UPSAMPLED_SIGNAL_LENGTH) for i in range(N_objects)])

delta8_fn = lambda :np.array([generate_random_object('delta', num_deltas=8, object_size=UPSAMPLED_SIGNAL_LENGTH) for i in range(N_objects)])


# plot an example of each object
wn = white_noise_fn()[0]
delta = delta_fn()[0]
delta8 = delta8_fn()[0]

KeyboardInterrupt: 

In [12]:


for replicate in range(num_replicates):

  mi_by_object = {}
  mi_by_object_hi = {}
  mi_by_object_lo = {}
  for name in object_names:
    print(name)
    mutual_information = []
    mi_hi = []
    mi_lo = []
    for num_nyquist_samples in tqdm(num_nyquist_samples_list):
      
      if name == 'delta_uniform':
        objects_fn = delta_fn
      elif name == '8_deltas_uniform':
        objects_fn = delta8_fn
      else:
        objects_fn = lambda : white_noise_fn()

      
      initial_kernel, initial_params, optimized_params, objects, initial_mi, optimized_mi, optimized_mi_lower, optimized_mi_upper = optimize_PSF_and_estimate_mi(
            objects_fn, noise_sigma, num_nyquist_samples=num_nyquist_samples, upsampled_signal_length=UPSAMPLED_SIGNAL_LENGTH,

            estimate_with_pixel_cnn=True,

            max_epochs=max_epochs, confidence=confidence)

      mutual_information.append(optimized_mi)
      mi_lo.append(optimized_mi_lower)
      mi_hi.append(optimized_mi_upper)
    mi_by_object[name] = mutual_information
    mi_by_object_hi[name] = mi_hi
    mi_by_object_lo[name] = mi_lo


  fig, ax = plt.subplots(1, 1, figsize=(3.5, 3.5))

  for object_name, mutual_information in mi_by_object.items():

      total_mi = np.array(mutual_information) * np.array(num_nyquist_samples_list)

      total_mi_hi = np.array(mi_by_object_hi[object_name]) * np.array(num_nyquist_samples_list)
      total_mi_lo = np.array(mi_by_object_lo[object_name]) * np.array(num_nyquist_samples_list)

      ax.plot(num_nyquist_samples_list, total_mi, 'o-', label=object_name)
      ax.fill_between(num_nyquist_samples_list, total_mi_lo, total_mi_hi, alpha=0.2)
      ax.set_xlabel('Signal bandwidth (number of Nyquist samples)')
      ax.set_ylabel('Mutual information (bits)')

      ax.set(ylim=(0, max(1.1 * np.max(total_mi), ax.get_ylim()[1])))
      # ax.set(ylim=[0, 25])

  clear_spines(ax)
  ax.legend()

  fig.savefig('/home/hpinkard_waller/figures/1d_signals/' + f'mi_vs_bandwidth_replicate{replicate}' + '.pdf', transparent=True)


white_noise


  0%|          | 0/5 [00:00<?, ?it/s]

initial loss -1.9429441095615134


  0%|          | 0/5 [02:17<?, ?it/s]


KeyboardInterrupt: 